In [1]:
import pandas as pd
import scipy as sp 
import numpy as np
import itertools

import csv

%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib

import graphlab as gl

from sets import Set
import re

import urlparse
import datetime as dt

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
[WARNING] Unable to write current GraphLab Create license to /home/usr/.graphlab/config. Ensure that this user account                         has write permission to /home/usr/.graphlab/config to save the license for offline use.
[INFO] GraphLab Create v1.8.3 started. Logging: /tmp/graphlab_server_1459793615.log
[WARNING] Unable to create session in specified location: '/home/usr/.graphlab/artifacts'. Using: '/var/tmp/graphlab-usr/4072/tmp_session_45001d36-7a2a-46b9-b2c6-4555e19e5f18'
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: DeprecationWarning: the sets module is deprecated


Для анализа был предоставлен файл на ~ 10 млн. строк

wc -l rfm.tsv
10007147 rfm.tsv

In [2]:
# timestamp хита 
# user_id 
# url - url хита или "click", если пользователь перешел в интернет-магазин 
# geo_id - идентификатор региона пользователя 
# category - идентификатор категории 
# price

h = ["timestamp", "user_id", "url", "geo_id", "category", "price"]

# df = pd.read_csv('rfm_tail.tsv', sep="\t", index_col=None, header=None, nrows=50)
# т.к пандас быстро не заработал с такими данными, то попробуем по-другому

In [63]:
df = []

limit = 10007148

with open('rfm.tsv', 'rb') as csvfile:
    r = csv.reader(csvfile, delimiter='\t')
    n = 0 
    for row in r:
        if n < limit: 
            df.append(row)
            n+=1
        else:
            break
        
df = pd.DataFrame.from_records(df, index=None, exclude=None, 
                                columns=h, 
                                coerce_float=False, nrows=None)

df.fillna(value=np.nan, inplace=True)

In [76]:
df.shape[0]

10007147

In [5]:
df[490000:490003]

,timestamp,user_id,url,geo_id,category,price
490000,1361989322,0cf1661de61fc270872dbf008528ce5c,/model.xml?modelid=8483248&hid=91259,10000,91259,NaN
490001,1361563817,0cf1661de61fc270872dbf008528ce5c,/model.xml?modelid=8483281&hid=91259,10000,91259,NaN
490002,1361563663,0cf1661de61fc270872dbf008528ce5c,/model.xml?modelid=8483284&hid=91259,10000,91259,NaN


В процессе обработки обнаружилось, что в столбце с категориями данные грязные.

Считаем на всей выборке и экспортируем плохие категории в файл, чтобы посмотреть.

In [78]:
# результаты расчета в categories_fails.txt

categories = df["category"].unique()[1:]

with open("categories.txt", 'w') as o:
    for category in categories:
        o.write(category+"\n")

rx1 = re.compile("^.*[^0-9].*$")

def check_categories(cat, rx=rx1):    
    if type(cat)==str:
        match = rx.match(cat)
        if match:
            return match.group(0)
        else:
            return np.nan
    else:
        return np.nan
        
df["category_fail"] = df["category"].apply(check_categories)

with open("categories_fails_.txt", 'w') as o:
    for category in df["category_fail"].unique():
        o.write(str(category)+"\n")
        
with open("categories_fails.txt", 'rb') as f:
    for line in f:
        print line

df.drop("category_fail", 1, inplace=True)

nan

None

null

NaN

undefined

90589класс

138608#e9b

138608#e9btp70nmi70cv

90594#e9b

90594#Kv7bm1m3sd9s

90639#e9b

90639#HD-zmn8v0

90639#X9vfmdv1q

90564#e9b

90564#e9btbx1s7wmnd9av

90564#e9btpwmna9

90796#e9b

90796#qskvd9j8t7a1v2xcvd95

 226667

91107http://market.yandex.ru/model.xml?modelid=2212950

9101315:46:03

6269371/

91028/

91031.

91/ru-BY

91/ru-KZ

91/ru-UA

90 548

91148.

734595/

91491clid=523

90639,

90594/

6427100/

91​491

90639￼

91491MSSQLAndDotNet

91031 или http://market.yandex.ru/model.xml?modelid=8232101

835...er=8464875:8000

90617Вот

 7815023

91013>>356306

91031-

6427100#tab=grades

91529  4900 ��� ��� ������� ������ 20

91259,а

91259,но

6427100a

914jm,91

91019-VIS=70-CAT_ID=651600-EXC=1-PG=10

91013.

91028http://

91491#A-GPS

91491#Bluetooth

91491#Dm0mav8snv

91491#Dpdao95tws7bnmcm2dmitjvn52a9

91491#DVB-H

91491#e9b

91491#e9bt7sd7mndmfmt4anvdv

91491#e9btSIM-avn0x

91491#EDGE

91491#eG-0edsntovdv1mfmcxip

91491#es1szmdt215tbm391xkt1

В общем, конечно можно просто спарсить категории заново из параметра hid как-то так, но это вроде вне задачи пока
так что на время забиваем.

In [79]:
def get_param(url, name):
    url_parsed = urlparse.urlparse(url)    
    qw = url_parsed.query    
    params = urlparse.parse_qs(url_parsed.query)    
    param = params.get(name)
    
    if param:
        return param[0]
    else:
        return param
    

urls_ex = df["url"].unique()[:100]
args_ = itertools.izip(urls_ex, itertools.repeat("hid", len(urls_ex)))


params = [get_param(*item) for item in args_]
params[:5]

[None, None, None, '7811975', '7811975']

In [80]:
urls_ = df["url"].tolist()
           
args_ = itertools.izip(urls_, itertools.repeat("hid", len(urls_)))

df["category_from_url"] = [get_param(*item) for item in args_]

In [81]:
df["category_fail_2"] = df["category_from_url"].apply(check_categories)

bad_urls = df[df["category_fail_2"].notnull()]
        
with open("categories_fails_2.txt", 'w') as o:
    for category in df["category_fail_2"].unique():
        o.write(str(category)+"\n")

In [65]:
df["price"].fillna(-1,inplace=True)
df["price"] = df["price"].astype(int)
df["price"].replace(-1, np.nan, inplace=True)
df["price"].replace(0, np.nan, inplace=True) # грязно хакаем нулевую цену, чтобы она не мешала в расчетах
df["price"].describe()

count    363181.000000
mean         28.240646
std          51.870724
min           5.000000
25%          10.000000
50%          10.000000
75%          20.000000
max        1000.000000
Name: price, dtype: float64

In [66]:
df["price"].sum()

10256466.0

In [87]:
def click_id(val):
    if np.isnan(val):
        return np.nan
    else:
        return 1

    
df["click_id"] = df["price"].apply(click_id)
df[df["click_id"].notnull()][:5]

,timestamp,user_id,url,geo_id,category,price,click_id,category_from_url,category_fail_2
64,2013-03-16 10:41:49,00023fcc5c93cb4674bdad0d448272e6,click,213,2662954,30.0,1.0,NaN,NaN
65,2013-03-13 10:09:18,00023fcc5c93cb4674bdad0d448272e6,click,213,6427100,10.0,1.0,NaN,NaN
66,2013-03-14 10:17:30,00023fcc5c93cb4674bdad0d448272e6,click,213,6427100,10.0,1.0,NaN,NaN
67,2013-03-14 10:26:10,00023fcc5c93cb4674bdad0d448272e6,click,213,6427100,10.0,1.0,NaN,NaN
68,2013-03-13 13:07:52,00023fcc5c93cb4674bdad0d448272e6,click,213,6427100,80.0,1.0,NaN,NaN


In [68]:
now = dt.datetime.now()
now

datetime.datetime(2016, 4, 4, 22, 25, 1, 527645)

In [69]:
df["timestamp"] = df["timestamp"].astype(int)
df["timestamp"] = df["timestamp"].apply(dt.datetime.fromtimestamp)

In [70]:
df["timestamp"].max(), df["timestamp"].min()

(Timestamp('2013-12-31 23:58:37'), Timestamp('2013-01-01 00:00:53'))

In [71]:
rfmTable = df.groupby('user_id').agg({
                                        'timestamp' : lambda x: (now - x.max()).days, #R
                                        'click_id': lambda x: x.sum(), #F
                                        'price': lambda x: x.sum() #M
                                    })

In [72]:
rfmTable["timestamp"] = rfmTable['timestamp'].astype(int)

In [77]:
rfmTable.to_csv("rfm_users.csv", sep=";")
rfm_sorted = rfmTable[rfmTable["price"].notnull()].sort("timestamp")
rfm_sorted.to_csv("rfm_sorted", sep=";")

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [97]:
rfmTable[rfmTable["price"].notnull()].sort("timestamp")[:10]

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,click_id,timestamp,price
user_id,,,
f120b615df2e1a8bcf6d793bdbbd503f,2.0,824,20.0
7a42e033e5cb95a03f689e822a16ad9b,1.0,824,209.0
8eae1c45d9195b869b255285c0e2da6d,4.0,824,127.0
3f51be06a90937eb91d1a6dc7fd67049,3.0,824,28.0
580268437067e3646b189a6647d40c3b,1.0,824,56.0
86c537f5d1251c644fed9f71360eac35,9.0,824,104.0
170f725b0a30a91e077950cf94127e4b,16.0,824,310.0
5c95466099af23021422627fe81dcebd,1.0,824,10.0
8a7a55cf275351bfcb241bb176c8a530,1.0,824,10.0


In [90]:
df["category_from_url"].fillna(np.nan,inplace=True)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [95]:
df.loc[df['category_fail_2'].notnull(), 'category'] = -1

In [96]:
df[df['category_fail_2'].notnull()]

,timestamp,user_id,url,geo_id,category,price,click_id,category_from_url,category_fail_2
7144,2013-03-30 20:46:15,003af13e1a49613fbbbcb9d6f4fbc546,"/gate/filter-modelmeter.xml?CMD=-RR=9,0,0,0-PF...",10000,-1,NaN,NaN,null,null
7145,2013-03-30 20:47:25,003af13e1a49613fbbbcb9d6f4fbc546,"/gate/filter-modelmeter.xml?CMD=-RR=9,0,0,0-PF...",10000,-1,NaN,NaN,null,null
7146,2013-03-30 20:47:22,003af13e1a49613fbbbcb9d6f4fbc546,"/gate/filter-modelmeter.xml?CMD=-RR=9,0,0,0-PF...",10000,-1,NaN,NaN,null,null
7147,2013-03-30 20:43:16,003af13e1a49613fbbbcb9d6f4fbc546,"/gate/filter-modelmeter.xml?CMD=-RR=9,0,0,0-VI...",10000,-1,NaN,NaN,null,null
7148,2013-03-30 20:59:45,003af13e1a49613fbbbcb9d6f4fbc546,/gate/filter-modelmeter.xml?hid=null&CMD=-RR=9...,10000,-1,NaN,NaN,null,null
7157,2013-03-30 20:47:40,003af13e1a49613fbbbcb9d6f4fbc546,"/guru.xml?CMD=-RR=9,0,0,0-PF=2139796713~EQ~sel...",10000,-1,NaN,NaN,null,null
7158,2013-03-30 20:59:49,003af13e1a49613fbbbcb9d6f4fbc546,"/guru.xml?CMD=-RR=9,0,0,0-PF=2139796713~EQ~sel...",10000,-1,NaN,NaN,null,null
7159,2013-03-30 20:43:20,003af13e1a49613fbbbcb9d6f4fbc546,"/guru.xml?CMD=-RR=9,0,0,0-PF=2141002946~TR~sel...",10000,-1,NaN,NaN,null,null
7160,2013-03-30 20:45:20,003af13e1a49613fbbbcb9d6f4fbc546,"/guru.xml?CMD=-RR=9,0,0,0-PF=2141002946~TR~sel...",10000,-1,NaN,NaN,null,null
7164,2013-03-30 20:48:11,003af13e1a49613fbbbcb9d6f4fbc546,"/guru.xml?hid=null&CMD=-RR=9,0,0,0-PF=21397967...",10000,-1,NaN,NaN,null,null


In [98]:
rfmTable = df.groupby(['user_id', 'category']).agg({
                                        'timestamp' : lambda x: (now - x.max()).days, #R
                                        'click_id': lambda x: x.sum(), #F
                                        'price': lambda x: x.sum() #M
                                    })

rfmTable["timestamp"] = rfmTable['timestamp'].astype(int)

rfmTable.to_csv("rfm_users_categories.csv", sep=";")
rfm_sorted = rfmTable[rfmTable["price"].notnull()].sort("timestamp")
rfm_sorted.to_csv("rfm_sorted", sep=";")


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [101]:
df.to_csv("frame.csv", sep="~")

KeyboardInterrupt: 